In [ ]:
!pip install presidio_analyzer presidio_anonymizer
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

In [ ]:
text_to_anonymize = "My name is John Doe, you can reach me at 555-123-4567 or via email at john.doe@example.com. My company ID is 12345"

In [ ]:
analyzer = AnalyzerEngine()
analyzer_results = analyzer.analyze(text = text_to_anonymize, entities=["PHONE_NUMBER", "PERSON", "EMAIL_ADDRESS"], language='en')

print(analyzer_results)

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


[type: EMAIL_ADDRESS, start: 70, end: 90, score: 1.0, type: PERSON, start: 11, end: 19, score: 0.85, type: PHONE_NUMBER, start: 41, end: 53, score: 0.4]


In [ ]:
for result in analyzer_results:
  print(text_to_anonymize[result.start:result.end], result.entity_type)

john.doe@example.com EMAIL_ADDRESS
John Doe PERSON
555-123-4567 PHONE_NUMBER


In [ ]:
from presidio_analyzer import Pattern, PatternRecognizer

id_pattern = Pattern(name="id_pattern", regex="\d{5}", score=0.5)

id_recognizer = PatternRecognizer(
    supported_entity="ID",
    patterns=[id_pattern],
    supported_language="en"
)

analyzer.registry.add_recognizer(id_recognizer)

In [ ]:
analyzer_results = analyzer.analyze(text = text_to_anonymize, entities=["PHONE_NUMBER", "PERSON", "EMAIL_ADDRESS", "ID"], language='en')

print(analyzer_results)

[type: EMAIL_ADDRESS, start: 70, end: 90, score: 1.0, type: PERSON, start: 11, end: 19, score: 0.85, type: ID, start: 109, end: 114, score: 0.5, type: PHONE_NUMBER, start: 41, end: 53, score: 0.4]


In [ ]:
for result in analyzer_results:
  print(text_to_anonymize[result.start:result.end], result.entity_type)

john.doe@example.com EMAIL_ADDRESS
John Doe PERSON
12345 ID
555-123-4567 PHONE_NUMBER


In [ ]:
anonymizer = AnonymizerEngine()

anonymized_results = anonymizer.anonymize(
    text=text_to_anonymize,
    analyzer_results=analyzer_results
)

print (f"text : {anonymized_results.text}")

text : My name is <PERSON>, you can reach me at <PHONE_NUMBER> or via email at <EMAIL_ADDRESS>. My company ID is <ID>


In [ ]:
from presidio_anonymizer.entities import OperatorConfig

operators={"PHONE_NUMBER": OperatorConfig("mask", {"type": "mask", "masking_char": "*", "chars_to_mask": 12, "from_end": True}),
            "DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"})}


custom_anonymized_results = anonymizer.anonymize(
    text=text_to_anonymize,
    analyzer_results=analyzer_results,
    operators = operators
)

print (f"text : {custom_anonymized_results.text}")

text : My name is <ANONYMIZED>, you can reach me at ************ or via email at <ANONYMIZED>. My company ID is <ANONYMIZED>
